In [ ]:
from glob import glob
from gzip import open # NB: overrides standard open()
import os
import math
import functools
import numpy as np
import scipy.interpolate
from scipy import interpolate
import pandas as pd
import pickle as pkl

In [ ]:
def is_nan(x):
    return (x is np.nan or x != x)

In [ ]:
# y = the regressors, a simple 2D array
y =pd.read_csv('C:/Users/admin/Desktop/курсовая 2.0/y.txt', sep= '\t')
y = y.replace(',','.', regex=True) #меняю ',' на '.'
y = y.astype(float) #конвертирую из string в float
y=y.assign(Humic=y[["Pahokee peat","Nordic aquatic","Suwanee river","Elliot soil","Disser"]].sum(1)) #складываю столбцы
y=y.drop(["Pahokee peat","Nordic aquatic","Suwanee river","Elliot soil","Disser"], axis=1) #удаляю ненужные столбцы
y = y[['Humic'] + y.columns[:-1].tolist()] #меняю порядок столбцов так, чтобы первым был Humic
y.to_pickle('C:/Users/admin/Desktop/курсовая 2.0/y.pkl.gz') #сохраняем в pickle формате, удобном для регрессии
pd.set_option('display.float_format', '{:.15f}'.format) #контроль длинны вывода float
print()

# save the result
y.to_pickle('C:/Users/admin/Desktop/курсовая 2.0/y.pkl.gz')

In [ ]:
# X = the predictors, a tensor
# dimensions: (sample, emission wavelength, excitation wavelength)

# first read the text files individually
# make sure to sort the list of files, they must correspond to rows in y
X_ = [pd.read_csv(f, sep='\t') for f in sorted(glob('C:/Users/admin/Desktop/курсовая 2.0/X — копия/??.txt'))]
# sanity check: wavelengths must be the same in all files
for x in X_[1:]:
	# errors will also happen if any of the files have different dimensions
	if not ((x.axes[0] == X_[0].axes[0]).all() and (x.axes[1] == X_[0].axes[1]).all()):
		# it doesn't quite matter what we raise here
		# the important thing is to complain loudly and crash
		raise Exception(x.axes)
pd.set_option('display.float_format', '{:.15f}'.format) #контроль длинны вывода float
k = 0
X=np.zeros([35,206,55]) #инициализирую массив нулей

fds = sorted(os.listdir('C:/Users/admin/Desktop/курсовая 2.0/X — копия/')) #формируем список файлов x для считывания в цикле

xw = pd.read_csv('C:/Users/admin/Desktop/курсовая 2.0/X — копия/01.txt',sep='\t')
colums=list(xw.columns.values) #установил имена колонок
rows=list(xw.index.values)  #установил имена строчек
ll=xw.columns.values #колонки
lrl=xw.index.values #строчки
lrl=lrl.astype(float)
ll=ll.astype(float)
Xw=np.zeros([206,55]) #инициализирую массив нулей
def lo(t):
    h=(1/t)*10**9
    hh=h-340000
    return (1/hh)*10**9
for i in range(0,206):
    for j in range(0,55):
        if (math.fabs((2*ll[j])-(lrl[i]))<33) or (math.fabs(ll[j]-(lrl[i]))<18) or(math.fabs((2*lrl[i])-(lo(ll[j])))<33) or (math.fabs(lrl[i]-lo(ll[j]))<18):
            Xw[i,j]=1.0  #собираю булеву матрицу
        
            
print(lrl)
print(ll)
print(Xw)
print(lo(lrl[17]))
print(lrl[17])
while k < 35:
    x = pd.read_csv('C:/Users/admin/Desktop/курсовая 2.0/X — копия/'+fds[k],sep='\t')
    x = x.fillna(0)
    x = x.replace(',','.', regex=True) #меняю ',' на '.'
    x = x.astype(float) #конвертирую из string в float
    Xx=np.zeros([206,55]) #инициализирую массив нулей
    for gg in range(0,206):
        tt=[]  #будущий набор из названий строк для интерполяции
        f2=[]
        f3=[]
        f5=[]
        f6=[]
        f4=[]
        tt1=[]
        fgh=x.iloc[gg,:]
        gh=0
        kj=0
        lk=0
        km=0
        if(math.fabs(1-Xw[gg,0])>0.5) and (math.fabs(1-Xw[gg,6])>0.5):
            for i in range(0,6):
                if (i<4): #область которую буду интерполировать
                    tt1+=[ll[i]]
                    lk+=1
                if (i>=4): #область данных
                    f4+=[ll[i]]
                    f6+=[fgh.iloc[i]]
        else:
            for i in range(0,6):
                tt+=[ll[i]]
                
        for i in range(6,55):
            if (math.fabs(1-Xw[gg,i])<0.5): #область для интерполяции
                tt+=[ll[i]]
                gh+=1
            if (math.fabs(1-Xw[gg,i])>0.5):  #область для получения интерполяционного многочлена
                f2+=[fgh.iloc[i]]
                f3+=[ll[i]]
                kj+=1
            
               
        if (len(f3)>1) and (len(f2)>1):
            a1=scipy.interpolate.PchipInterpolator(f3,f2)
            a2=a1(tt).astype(float)
        else:
            a2=0.0
        if (len(f4)>1) and (len(f6)>1):
            a4=scipy.interpolate.PchipInterpolator(f4,f6)
            a3=a4(tt1).astype(float)
        vc=0
        cv=0 
        v=0
        v1=0
        for i in range(0,55):
            if (len(tt1)>0.5) and (v<len(tt1)):
                if (math.fabs(ll[i]-tt1[v])<1):
                    f5+=[a3[v]]
                    v+=1
            if (len(f4)>0.5) and (v1<len(f4)):
                if (math.fabs(ll[i]-f4[v1])<1):
                    f5+=[f6[v1]]
                    v1+=1
            
            if (len(f3)>0.5) and (vc<len(f3)):
                if (math.fabs(ll[i]-f3[vc])<1):
                    f5+=[f2[vc]]
                    vc+=1
            if (len(tt)>0.5) and (cv<len(tt)):
                if (math.fabs(ll[i]-tt[cv])<1):
                    f5+=[a2[cv]]
                    cv+=1
        Xx[gg,:]+=f5
    X[k,:,:]+=Xx #собираю тензор
    k = k + 1
# save the fluorescence intensity values together with wavelengths
# (like Matlab, NumPy doesn't seem to have the concept
# of "named dimensions" in >2-dimensional arrays)
pkl.dump(
	{'X': X, 'emission': X_[0].axes[0].values, 'excitation': X_[1].axes[1].values},
	open('C:/Users/admin/Desktop/курсовая 2.0/X.pkl.gz', 'wb')
)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[5,:,:],aspect="auto")
plt.show();
plt.imshow(x,aspect="auto")
plt.show();
plt.plot(ll,f5)
plt.show();
plt.plot(ll,fgh)
plt.show();
print()
print(X[5,:,:])
print(a2)


[240. 242. 244. 246. 248. 250. 252. 254. 256. 258. 260. 262. 264. 266.
 268. 270. 272. 274. 276. 278. 280. 282. 284. 286. 288. 290. 292. 294.
 296. 298. 300. 302. 304. 306. 308. 310. 312. 314. 316. 318. 320. 322.
 324. 326. 328. 330. 332. 334. 336. 338. 340. 342. 344. 346. 348. 350.
 352. 354. 356. 358. 360. 362. 364. 366. 368. 370. 372. 374. 376. 378.
 380. 382. 384. 386. 388. 390. 392. 394. 396. 398. 400. 402. 404. 406.
 408. 410. 412. 414. 416. 418. 420. 422. 424. 426. 428. 430. 432. 434.
 436. 438. 440. 442. 444. 446. 448. 450. 452. 454. 456. 458. 460. 462.
 464. 466. 468. 470. 472. 474. 476. 478. 480. 482. 484. 486. 488. 490.
 492. 494. 496. 498. 500. 502. 504. 506. 508. 510. 512. 514. 516. 518.
 520. 522. 524. 526. 528. 530. 532. 534. 536. 538. 540. 542. 544. 546.
 548. 550. 552. 554. 556. 558. 560. 562. 564. 566. 568. 570. 572. 574.
 576. 578. 580. 582. 584. 586. 588. 590. 592. 594. 596. 598. 600. 602.
 604. 606. 608. 610. 612. 614. 616. 618. 620. 622. 624. 626. 628. 630.
 632.